## How to read meteorogical data with `read_mdf` function

In [1]:
from __future__ import annotations

import os
import sys

import pandas as pd
import polars as pl

from cdm_reader_mapper import properties, read_mdf, test_data

2025-01-31 09:29:23,553 - root - INFO - init basic configure of logging success
/Users/josidd/git/github/cdm_fork/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The `cdm_reader_mapper.read_mdf` function and is a tool designed to read data files compliant with a user specified [data
model](https://cds.climate.copernicus.eu/toolbox/doc/how-to/15_how_to_understand_the_common_data_model/15_how_to_understand_the_common_data_model.html).

It was developed with the initial idea to read the [IMMA](https://icoads.noaa.gov/e-doc/imma/R3.0-imma1.pdf) data format, but it was further enhanced to account for other meteorological data formats.

Lets see an example for a typical file from [ICOADSv3.0.](https://icoads.noaa.gov/r3.html). We pick an specific monthly output for a Source/Deck. In this case data from the Marine Meterological Journals data set SID/DCK: **125-704 for Oct 1878.**

The `.imma` file looks like this:

In [2]:
data_path = test_data.test_icoads_r300_d704["source"]

data_ori = pd.read_table(data_path)
data_ori.head()

2025-01-31 09:29:27,576 - root - INFO - Attempting to fetch remote file: icoads/r300/d704/input/icoads_r300_d704_1878-10-01_subset.imma.md5


,"18781020 600 4228 29159 130623 10Panay 12325123 9961 4 165 17128704125 5 0 1 1FF111F11AAA1AAAA1AAA 9815020N163002199 0 100200180003Panay 78011118737S.P.Bray,Jr 013231190214 Bulkhead of cabin 1- .1022200200180014Boston Rio de Janeiro 300200180014001518781020 4220N 6630W 10 E 400200180014001518781020102 85 EXS WSW 0629601 58 BOC CU05R"
0,18781020 800 4231 29197 130623 10Panay 1...
1,187810201000 4233 29236 130623 10Panay 1...
2,187810201200 4235 29271 130623 10Panay 1...
3,187810201400 4237 29310 130623 10Panay 1...


Very messy to just read into python!

This is why we need the `mdf_reader` tool, to helps us put those imma files in a [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) format. For that we need need a **schema**.

A **schema** file gathers a collection of descriptors that enable the `mdf_reader` tool to access the content
of a `data model/ schema` and extract the sections of the raw data file that contains meaningful information. These **schema files** are the `bones` of the data model, basically `.json` files outlining the structure of the incoming raw data.

The `mdf_reader` takes this information and translate the characteristics of the data to a python polars dataframe.

The tool has several **schema** templates build in.

In [3]:
properties.supported_data_models

['craid', 'gcc', 'icoads', 'pub47']

**Schemas** can be designed to be deck specific like the example below

In [4]:
schema = "icoads_r300_d704"

data = read_mdf(data_path, imodel=schema, validate=False)

2025-01-31 09:29:27,710 - root - INFO - READING DATA MODEL SCHEMA FILE...
2025-01-31 09:29:27,718 - root - INFO - EXTRACTING DATA FROM MODEL: icoads_r300_d704
2025-01-31 09:29:27,718 - root - INFO - Getting data string from source...
2025-01-31 09:29:27,789 - root - INFO - Extracting and reading sections
2025-01-31 09:29:27,875 - root - INFO - Create output DataBundle object


A new **schema** can be build for a particular deck and source as shown in this notebook. The `imma1_d704` schema was build upon the `imma1` schema/data model but extra sections have been added to the `.json` files to include supplemental data from ICOADS documentation. This is a snapshot of the data inside the `imma1_d704.json`.

```
"c99_journal": {
            "header": {"sentinal": "1", "field_layout":"fixed_width","length": 117},
            "elements": {
              "sentinal":{
                  "description": "Journal header record identifier",
                  "field_length": 1,
                  "column_type": "str"
              },
              "reel_no":{
                  "description": "Microfilm reel number. See if we want the zero padding or not...",
                  "field_length": 3,
                  "column_type": "str",
                  "LMR6": true
              }
            ...
```

The full output `polars.DataFrame` can be accessed:

In [5]:
data.data

index,_core_missing,core:YR,core:MO,core:DY,core:HR,core:LAT,core:LON,core:IM,core:ATTC,core:TI,core:LI,core:DS,core:VS,core:NID,core:II,core:ID,core:C1,core:DI,core:D,core:WI,core:W,core:VI,core:VV,core:WW,core:W1,core:SLP,core:A,core:PPP,core:IT,core:AT,core:WBTI,core:WBT,core:DPTI,core:DPT,core:SI,core:SST,…,c99_data4:sea_temperature,c99_data4:present_weather,c99_data4:clouds,c99_data4:sky_clear,c99_data4:sea_state,_c99_data5_missing,c99_data5:sentinal,c99_data5:reel_no,c99_data5:journal_no,c99_data5:frame_start,c99_data5:frame,c99_data5:year,c99_data5:month,c99_data5:day,c99_data5:time_ind,c99_data5:hour,c99_data5:ship_speed,c99_data5:compass_ind,c99_data5:ship_course_compass,c99_data5:blank,c99_data5:ship_course_true,c99_data5:wind_dir_mag,c99_data5:wind_dir_true,c99_data5:wind_force,c99_data5:barometer,c99_data5:temp_ind,c99_data5:attached_thermometer,c99_data5:air_temperature,c99_data5:wet_bulb_temperature,c99_data5:sea_temperature,c99_data5:present_weather,c99_data5:clouds,c99_data5:sky_clear,c99_data5:sea_state,c99_data5:compass_correction_ind,c99_data5:compass_correction,c99_data5:compass_correction_dir
u32,bool,i64,i64,i64,f64,f64,f64,str,i64,str,str,str,str,str,str,str,str,str,i64,str,f64,str,str,str,str,f64,str,f64,str,f64,str,f64,str,f64,str,f64,…,f64,str,str,i64,str,bool,str,str,str,str,str,i64,i64,i64,str,i64,f64,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,str,str,i64,str,str,f64,str
0,false,1878,10,20,6.0,42.28,291.59,"""1""",3,"""0""","""6""","""2""","""3""",null,"""10""","""Panay""",null,"""1""",232,"""5""",12.3,null,null,null,null,996.1,null,null,null,null,null,null,null,null,null,null,…,null,"""BOC""","""CU""",5,"""R""",true,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1,false,1878,10,20,8.0,42.31,291.97,"""1""",3,"""0""","""6""","""2""","""3""",null,"""10""","""Panay""",null,"""1""",232,"""5""",12.3,null,null,null,null,996.3,null,null,null,null,null,null,null,null,null,null,…,null,"""BOC""","""SC""",3,"""R""",true,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2,false,1878,10,20,10.0,42.33,292.36,"""1""",3,"""0""","""6""","""2""","""3""",null,"""10""","""Panay""",null,"""1""",254,"""5""",12.3,null,null,null,null,996.9,null,null,"""7""",8.9,null,null,null,null,"""1""",11.1,…,5.2,"""OCG""","""SC""",0,"""R""",true,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3,false,1878,10,20,12.0,42.35,292.71,"""1""",3,"""0""","""6""","""2""","""3""",null,"""10""","""Panay""",null,"""1""",254,"""5""",12.3,null,null,null,null,997.6,null,null,"""7""",8.9,null,null,null,null,"""1""",11.1,…,5.2,"""CG""","""SC""",0,"""R""",true,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
4,false,1878,10,20,14.0,42.37,293.1,"""1""",3,"""0""","""6""","""2""","""3""",null,"""10""","""Panay""",null,"""1""",254,"""5""",12.3,null,null,null,null,999.2,null,null,"""7""",8.9,null,null,null,null,"""1""",10.0,…,5.0,"""BC""","""SC""",2,"""L""",true,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


Now metadata information can be extracted as a component of the polars dataframe.

In [6]:
data.data.select(pl.col("^c99_journal:.*$"))

c99_journal:sentinal,c99_journal:reel_no,c99_journal:journal_no,c99_journal:frame_no,c99_journal:ship_name,c99_journal:journal_ed,c99_journal:rig,c99_journal:ship_material,c99_journal:vessel_type,c99_journal:vessel_length,c99_journal:vessel_beam,c99_journal:commander,c99_journal:country,c99_journal:screw_paddle,c99_journal:hold_depth,c99_journal:tonnage,c99_journal:baro_type,c99_journal:baro_height,c99_journal:baro_cdate,c99_journal:baro_loc,c99_journal:baro_units,c99_journal:baro_cor,c99_journal:thermo_mount,c99_journal:SST_I
str,str,str,str,str,str,str,str,str,i64,i64,str,str,str,i64,i64,str,i64,str,str,str,str,str,str
"""1""","""002""","""0018""","""0003""","""Panay""","""78""","""01""","""1""","""1""",187,37,"""S.P.Bray,Jr""","""01""","""3""",23,1190,"""2""",14,null,"""Bulkhead of cabin""","""1""","""- .102""","""2""",null
"""1""","""002""","""0018""","""0003""","""Panay""","""78""","""01""","""1""","""1""",187,37,"""S.P.Bray,Jr""","""01""","""3""",23,1190,"""2""",14,null,"""Bulkhead of cabin""","""1""","""- .102""","""2""",null
"""1""","""002""","""0018""","""0003""","""Panay""","""78""","""01""","""1""","""1""",187,37,"""S.P.Bray,Jr""","""01""","""3""",23,1190,"""2""",14,null,"""Bulkhead of cabin""","""1""","""- .102""","""2""",null
"""1""","""002""","""0018""","""0003""","""Panay""","""78""","""01""","""1""","""1""",187,37,"""S.P.Bray,Jr""","""01""","""3""",23,1190,"""2""",14,null,"""Bulkhead of cabin""","""1""","""- .102""","""2""",null
"""1""","""002""","""0018""","""0003""","""Panay""","""78""","""01""","""1""","""1""",187,37,"""S.P.Bray,Jr""","""01""","""3""",23,1190,"""2""",14,null,"""Bulkhead of cabin""","""1""","""- .102""","""2""",null


To learn how to construct a schema or data model for a particular deck/source, visit this other [tutorial notebook](https://github.com/glamod/cdm_reader_mapper/blob/main/docs/example_notebooks/CLIWOC_datamodel.ipynb)